# Imports

In [1]:
# imports
from PyPDF2 import PdfFileMerger

import math

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import pyplot as plt
from matplotlib import colors
# MatPlotLib tools for drawing on plots.
import matplotlib.transforms as mtransforms
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

import numpy as np

import pandas as pd

import pickle

import os        

import glob 

import sys

from datetime import datetime

import seaborn as sns

import tables


# Scipy

import scipy.stats
from scipy.optimize import fmin
import scipy.optimize as opt
from scipy.interpolate import interp1d
from scipy.interpolate import LSQBivariateSpline, LSQUnivariateSpline
from scipy.optimize import least_squares
from scipy.signal import chirp, find_peaks, peak_widths
from scipy import signal
from scipy.optimize import curve_fit



#Astropy
import astropy.units as u
from astropy.io import fits
from astropy.stats import sigma_clip
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.units import cgs as c
from astropy.cosmology import WMAP5, WMAP7
from astropy.visualization import quantity_support
from astropy.io import ascii
from astropy.table import Table
from astropy.table import QTable
from astropy.time import Time
from astropy.visualization import quantity_support
from astropy.wcs import WCS
from astropy.modeling import models, fitting


from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude, get_constellation  # Angles

#Astroquery
from astroquery.simbad import Simbad
from astroquery.ned import Ned
from astroquery.vizier import Vizier

from specutils import Spectrum1D, SpectralRegion
from specutils.manipulation import noise_region_uncertainty
from specutils.fitting import find_lines_threshold
from specutils.fitting import find_lines_derivative
from specutils.fitting import fit_generic_continuum
from specutils.analysis import equivalent_width
from specutils.manipulation import FluxConservingResampler, LinearInterpolatedResampler, SplineInterpolatedResampler
from specutils.analysis import correlation

import warnings

In [2]:
plt.rc('axes', labelsize=14)
plt.rc('axes', labelweight='bold')
plt.rc('axes', titlesize=16)
plt.rc('axes', titleweight='bold')
plt.rc('font', family='sans-serif')
plt.rcParams['figure.figsize'] = (15, 7)
plt.rcParams['axes.grid'] = True

# Functions

In [3]:
def get_files(path, file_type = "*.fits"):
    files = [f for f in glob.glob(path+file_type)]
    files.sort()
    return files

In [4]:
#Select Feature:
#
#              Input_params[absorption_line, feature_min, feature_max, left_cont_min, 
#                           left_cont_max, right_cont_min, right_cont_max]
#             
#             absorption_line = Midpoint of absorption feature (used to distinguish specific line)
#             feature_min, feature_max =  Endpoints of the absorption feature 
#             left_cont_min, left_cont_max = Continuum Endpoints to left of feature
#             right_cont_min, right_cont_max = Continuum Endpoints to right of feature

def set_feature_params(line_info, Absorption_line_dict):
    #Line_info order: 
    #                absorption_line, 
    #                feature_min, feature_max,  
    #                left_cont_min, left_cont_max, 
    #                right_cont_min, right_cont_max
    
    absorption_line = line_info[0]
    feature_min, feature_max = line_info[1], line_info[2]

    left_cont_min, left_cont_max  = line_info[3], line_info[4]
    right_cont_min, right_cont_max = line_info[5], line_info[6]
    
    selected_absorption_line = str(absorption_line)
    

    line_params = { 'feature_min'    : feature_min,
                    'feature_max'    : feature_max,
                    'left_cont_min'  : left_cont_min,
                    'left_cont_max'  : left_cont_max,
                    'right_cont_min' : right_cont_min,
                    'right_cont_max' : right_cont_max }
        
    
    Absorption_line_dict[selected_absorption_line] = line_params


def feature_param_dict_to_pandas_df(Absorption_line_dict):
    df = pd.DataFrame.from_dict(Absorption_line_dict, orient='index')
    return df




In [5]:
#Model input data: 
#PHOENIX:
#        Model_Stellar_Params[Temp, log_g, M_H , alpha_M], Spectrum(Wavelength, Flux)
#        Temp = effective temperature (K)
#        log_g = log (surface gravity) (cm/s^2)
#        M_H = metallicity (rel. sol. – Asplund et al 2009)
#        alpha_M =  alpha element enhancement


def read_model_spec_file(spec_filename, wcs_filename, Modeltype = 'Phoenix'):
    #one file only, multple files handles in "model_input_data"
    
    if Modeltype == 'Phoenix':


        sp = fits.open(spec_filename)
        header = sp[0].header

        teff = header['PHXTEFF']
        log_g = header['PHXLOGG']
        M_H = header['PHXM_H']
        alpha_M = header['PHXALPHA']

        Stellar_params = np.array([log_g, M_H, alpha_M])

        wv = fits.open(wcs_filename)
        wcs = WCS(header)
        index = np.arange(header['NAXIS1'])

        wavelength = wcs.wcs_pix2world(index[:,np.newaxis], 0)
        wavelength = wavelength.flatten()
        flux = sp[0].data

    return wavelength, flux, teff, Stellar_params

#Constraint input order
#temperature min, temperature max, log_g, M_H ( [M/H] metallicity), alpha_M ([a/M] alpha element enhancement)

def model_input_data(Model_spectra_paths, Model_wcs_path, Constraints, Modeltype = 'Phoenix'):
    
    Model_Stellar_Params = {}
    Spectrum_data = {}
    Model_input_data = {}
    
    Wavelength = []
    Flux = []
    Teff = []

    
    if Modeltype == 'Phoenix':
        min_temp, max_temp, c_log_g, c_M_H, c_alpha_M = Constraints



        for f in Model_spectra_paths:
            wave, pf, pt, Stellar_params = read_model_spec_file(f, Model_wcs_path)
            log_g, m_h, alpha_M = Stellar_params

            if Constraints[0] == False:
                Flux.append(pf)
                Teff.append(pt)
                Wavelength.append(wave)
               


            else:

                if pt >= min_temp and pt <= max_temp and [log_g, m_h, alpha_M] == [c_log_g, c_M_H, c_alpha_M]:
                    Flux.append(pf)
                    Wavelength.append(wave)
                    Teff.append(pt)
                    


        

        
        Model_Stellar_Params['log_g'] = c_log_g
        Model_Stellar_Params['M_H'] = c_M_H
        Model_Stellar_Params['alpha_M'] = c_alpha_M
        
        
        Spectrum_data['wav']  = Wavelength
        Spectrum_data['flux'] = Flux
        Spectrum_data['Teff'] = Teff
        
        Model_input_data['Stellar_Params'] = Model_Stellar_Params
        Model_input_data['Spectrum_data'] = Spectrum_data
        
        
        
        #dtype=[('wave', 'f4'), ('Flux', 'f4'), 
                        #('Teff', 'f4'), ('mincont', 'f4'), ('maxcont', 'f4'), ('cont_range', 'f4')]

    return Model_input_data

In [6]:
#OBS input data:
#       Obs_Input_params[Spectrum, Epoch]
#       Spectrum = (Wavelength, Flux)
#       Epoch = 'YYYYMMDD'

def obs_input_data(MLO_spectra_path, epochs, source = 'MLO'):
        
    Spectrum_data = {}
    Obs_input_data = {}
    
    Wavelength = []
    Flux = []
    Epochs = Time(epochs, format='iso', out_subfmt='date')

    if source == 'MLO':
        for f in MLO_spectra_path:
            data = ascii.read(f, names=['wav', 'flux'])

            z_w = np.zeros(len(data['wav']))
            z_f = np.zeros(len(data['flux']))

            for i in np.arange(len(z_w)):
                z_w[i] = data['wav'][i]
            Wavelength.append(z_w)


            for i in np.arange(len(z_f)):
                z_f[i] = data['flux'][i]            
            Flux.append(z_f)
            
    Spectrum_data['wav']  = Wavelength
    Spectrum_data['flux'] = Flux
    
    Obs_input_data['epoch'] = Epochs
    Obs_input_data['Spectrum_data'] = Spectrum_data
      
    
    return Obs_input_data

In [7]:
def EQW(wave, flux, line_selection, line_dict, warnings_off = True):
    #If running well, able to turn off warnings to make program less annoying
    if warnings_off == True:
        warnings.filterwarnings("ignore")
            
    flux_units = (u.erg)*(u.cm**(-2))*(u.s**(-1))*(u.AA**(-1))
    
    ###INPUT###
    
    #Input Params: absorption line, feature_min, feature_max, 
    #              left_cont_min, left_cont_max, 
    #              right_cont_min, right_cont_max
    
    absorption_line = line_selection
    selected_absorption_line = str(line_selection)
    
    input_params = line_dict[selected_absorption_line]
    
    feature_min = input_params['feature_min']
    feature_max =input_params['feature_max']
    
    left_cont_min, left_cont_max   = input_params['left_cont_min'],  input_params['left_cont_max']
    right_cont_min, right_cont_max = input_params['right_cont_min'], input_params['right_cont_max']  
    
            

    
    #issolate aborption feature
    feature_cut = [(wave >= feature_min) & (wave <= feature_max)]
    
    #isolate continuum to left and right of absorption line 
    left_cont_cut  = [(wave >= left_cont_min)  &  (wave <= left_cont_max)]
    right_cont_cut = [(wave >= right_cont_min) &  (wave <= right_cont_max)]
    
    #Make a cut for easy viewing when plotting
    plot_cut =  [(wave >= feature_min - 150) & (wave <= feature_max + 150)]
    

    #Determine 'Knot Points' on left and right side of the feature
    #'Knot Points' set by median wavelength and flux values in the continuum ranges
    left_knot_x = np.median(wave[left_cont_cut])
    left_knot_y = np.median(flux[left_cont_cut])
    
    right_knot_x = np.median(wave[right_cont_cut])
    right_knot_y = np.median(flux[right_cont_cut])
    
    #Normalize feature by simple 'y = mx + b' line
    #2 versions of same line, one is extended for plotting purposes
    
    feat_only_cont_line = np.zeros(len(wave[feature_cut]))
    plot_cont_line = np.zeros(len(wave[plot_cut]))

    #find slope 'm' and y-intercept 'b'
    m = (left_knot_y-right_knot_y)/(left_knot_x-right_knot_x)
    #b = (x1*y2 - x2*y1)/(x1-x2)
    b = left_knot_y - m * left_knot_x
    
    #continuum line spanning absorption feature    
    for i in np.arange(len(wave[feature_cut])):
        x = wave[feature_cut][i]
        feat_only_cont_line[i] = m*x+b    
    
    #continuum line spanning plot
    for i in np.arange(len(wave[plot_cut])):
        x = wave[plot_cut][i]
        plot_cont_line[i] = m*x+b
    

    #Calculate EQW
    dw = wave[1] - wave[0]
    
    eqw = (dw) * np.abs(np.sum(1-(flux[feature_cut]/feat_only_cont_line)))
    
    #Plotting Data
    feature_cut_spectrum = {} 
    feature_cut_spectrum['x'], feature_cut_spectrum['y'], = wave[feature_cut], flux[feature_cut]/feat_only_cont_line
    feature_norm_cont_line   = {} 
    feature_norm_cont_line['x'], feature_norm_cont_line['y'], = wave[feature_cut], feat_only_cont_line/feat_only_cont_line
  
    cut_spectrum = {} 
    cut_spectrum['x'], cut_spectrum['y'], = wave[plot_cut], flux[plot_cut]
    norm_cont_line   = {} 
    norm_cont_line['x'], norm_cont_line['y'], = wave[plot_cut], plot_cont_line
    
    left_knot_point  = {} 
    left_knot_point['x'], left_knot_point['y'], = left_knot_x, left_knot_y #left knot point
    right_knot_point = {} 
    right_knot_point['x'], right_knot_point['y'], = right_knot_x, right_knot_y  #right knot point

    
    
    
    plotting_data = {}
    plotting_data['cut_spectrum'] = cut_spectrum
    plotting_data['feature_cut_spectrum'] = feature_cut_spectrum
    plotting_data['norm_cont_line'] = norm_cont_line
    plotting_data['feature_norm_cont_line'] = feature_norm_cont_line

    plotting_data['left_knot_point'] = left_knot_point
    plotting_data['right_knot_point'] = right_knot_point
    #[cut_spectrum, norm_cont_line, left_knot_point, right_knot_point]
    #plotting_data = pd.DataFrame[[cut_spectrum], [norm_cont_line], [left_knot_point], [right_knot_point]]
    #columns=['cut_spectrum', 'norm_cont_line', 'left_knot_point', 'right_knot_point']

    #[(wave[plot_cut], flux[plot_cut]), #spectrum (cut for plotting)
    #(wave[plot_cut], cont_line),  #Continuum line for notmalixing
    #(left_knot_x, left_knot_y), #left knot point
    #(right_knot_x, right_knot_y), #right knot point
    #(feature_min, flux[plot_cut].min(), flux[plot_cut].max()), #feature_min vertical line
    #(feature_max, flux[plot_cut].min(), flux[plot_cut].max()), #feature_max vertical line
    #(title)] #title
    
    return eqw, plotting_data

In [8]:
#Model Output data: 
#PHOENIX:
#        Model_Stellar_Params[Temp, log_g, M_H , alpha_M], Spectrum(Wavelength, Flux), (EQW_'Absorption_line')
#        EQW_'Absorption_line' = Equivalent Width of specific absorption line
def model_output_data(Model_input_data, line_selection, line_dict, model_output_dict):
    output_data = {}
    selected_output_data = {}

    data = {}

    ###INPUT###
    
    #Input Params: absorption line, feature_min, feature_max, 
    #              left_cont_min, left_cont_max, 
    #              right_cont_min, right_cont_max
    
    absorption_line = line_selection
    selected_absorption_line = str(line_selection)
    
    input_params = line_dict[selected_absorption_line]
    
    feature_min = input_params['feature_min']
    feature_max =input_params['feature_max']
    
    left_cont_min, left_cont_max   = input_params['left_cont_min'],  input_params['left_cont_max']
    right_cont_min, right_cont_max = input_params['right_cont_min'], input_params['right_cont_max']  

    
    #Input Data order: Wavelength, Spectra data, Temperature
    Spectrum_data = Model_input_data['Spectrum_data']
    
    wavelength = Spectrum_data['wav']
    flux = Spectrum_data['flux']
    
    temperatures = Spectrum_data['Teff']
    
    Model_Stellar_Params = Model_input_data['Stellar_Params']
    
    

    
    ### Run EQW Program ##
    eqw = []
    plotting_data = []
    
    
    for i in np.arange(len(flux)):
        eq, plot_dat = EQW(wavelength[i], flux[i], absorption_line, line_dict)
        eqw.append(eq)
        plotting_data.append(plot_dat)
        
    output_data['Stellar_Params'] = Model_Stellar_Params
    data['EQW'] = eqw
    data['Teff'] = temperatures
    output_data['data'] = data
    output_data['plotting_data'] = plotting_data
    
    #selected_output_data['EQW'] = eqw
    
    
    model_output_dict['Teff'] = temperatures
    model_output_dict[selected_absorption_line] = eqw
    
    
    
    return output_data

In [9]:
#OBS Output data:
#        Obs_Input_params[Spectrum, Epoch],  (EQW_'Absorption_line')
#        EQW_'Absorption_line' = Equivalent Width of specific absorption line
#        Epoch = 'YYYYMMDD'


def obs_output_data(Obs_input_data, line_selection, line_dict, Obs_output_dict):
    output_data = {}
    selected_output_data = {}

    data = {}

    ###INPUT###
    
    #Input Params: absorption line, feature_min, feature_max, 
    #              left_cont_min, left_cont_max, 
    #              right_cont_min, right_cont_max
    
    absorption_line = line_selection
    selected_absorption_line = str(line_selection)
    
    input_params = line_dict[selected_absorption_line]
    
    feature_min = input_params['feature_min']
    feature_max =input_params['feature_max']
    
    left_cont_min, left_cont_max   = input_params['left_cont_min'],  input_params['left_cont_max']
    right_cont_min, right_cont_max = input_params['right_cont_min'], input_params['right_cont_max']  

    
    #Input Data order: Wavelength, Spectra data, Temperature
    Spectrum_data = Obs_input_data['Spectrum_data']
    
    wavelength = Spectrum_data['wav']
    flux = Spectrum_data['flux']
    epochs = Obs_input_data['epoch']


    
    #Input Data order: Wavelength, Spectra data, Epochs ('YYYYMMDD')
    #Input Data order: Wavelength, Spectra data, Temperature

    
    
    ### Run EQW Program ##
        
    eqw = []
    plotting_data = []
    
    for i in np.arange(len(flux)):
        eq, plot_dat = EQW(wavelength[i], flux[i], absorption_line, line_dict)
        eqw.append(eq)
        plotting_data.append(plot_dat)
    
    output_data['epoch'] = epochs   
    data['EQW'] = eqw
    output_data['data'] = data
    output_data['plotting_data'] = plotting_data
    
    Obs_output_dict['Epoch'] = epochs
    
    
    
    Obs_output_dict[selected_absorption_line] = eqw
    return output_data

In [10]:
def knot_plot(output_data, line_selection, line_dict, output_path, nc = 2, model = False, save_fig = False):
    flux_units = (u.erg)*(u.cm**(-2))*(u.s**(-1))*(u.AA**(-1))
    

    
    ###INPUT###
    
    #Input Params: absorption line, feature_min, feature_max, 
    #              left_cont_min, left_cont_max, 
    #              right_cont_min, right_cont_max
    
    absorption_line = line_selection
    selected_absorption_line = str(line_selection)
    
    input_params = line_dict[selected_absorption_line]
    
    feature_min = input_params['feature_min']
    feature_max =input_params['feature_max']
    
    left_cont_min, left_cont_max   = input_params['left_cont_min'],  input_params['left_cont_max']
    right_cont_min, right_cont_max = input_params['right_cont_min'], input_params['right_cont_max']  

    #output_data: 'EQW', 'plotting_data', 'epochs', ('temperatures')
    data = output_data['data']
    eqw = data['EQW']
    plot_data = output_data['plotting_data']
    
    
    
    if model == False:
        epochs = output_data['epoch']
        knot_plot_filename = 'Obs_knot_plot_' + selected_absorption_line + ".pdf"
        selected_color = 'maroon'
    elif model == True:
        knot_plot_filename = 'Model_knot_plot_' + selected_absorption_line + ".pdf"
        temp = data['Teff']
        selected_color = 'navy'
    
    num_plots = len(output_data['plotting_data'])
    
    if (num_plots % 2) == 0:
        num_rows = math.floor(num_plots/nc)
    else:
        num_rows = math.floor(num_plots/nc + (num_plots)%nc)
    
    num_cols = nc
    
    fig, ax = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(30, 7*num_rows))
    if model == False:
        fig.suptitle('Knot Plot Observed Data')
    elif model == True:
        fig.suptitle('Knot Plot Data')
    
    for i in np.arange(num_rows):
        #Plotting_data: 'cut_spectrum', 'norm_cont_line', 'left_knot_point', 'right_knot_point'
        cut_spectrum = plot_data[i]['cut_spectrum']   
        norm_cont_line = plot_data[i]['norm_cont_line'] 
        left_knot_point = plot_data[i]['left_knot_point']
        right_knot_point = plot_data[i]['right_knot_point']
        feature_min_vertical_line = [feature_min, cut_spectrum['y'].min(), cut_spectrum['y'].max()]
        feature_max_vertical_line = [feature_max, cut_spectrum['y'].min(), cut_spectrum['y'].max()]
        
        if model == True:
            ax[i, 0].set_title(r'T = {}, EQW $\approx$ {}'.format(temp[i], np.round(eqw[i],0)))
        else:
            ax[i, 0].set_title(r'Date = {}, EQW $\approx$ {}'.format(epochs[i].jd, np.round(eqw[i],0)))
            

        ax[i, 0].plot(cut_spectrum['x'], cut_spectrum['y'], drawstyle='steps-mid', label='Initial Spectrum');  
        ax[i, 0].plot(norm_cont_line['x'], norm_cont_line['y'], label = 'Continuum Fit')
        ax[i, 0].scatter(left_knot_point['x'], left_knot_point['y'],  c = 'crimson', s = 100, label = 'Knot Points')
        ax[i, 0].scatter(left_knot_point['x'], left_knot_point['y'], c = 'crimson', s = 200)
        ax[i, 0].scatter(right_knot_point['x'], right_knot_point['y'], c = 'crimson', s = 200)
        ax[i, 0].vlines(feature_min_vertical_line[0], feature_min_vertical_line[1],  feature_min_vertical_line[2], colors='black', label = 'Feature Endpoints')
        ax[i, 0].vlines(feature_max_vertical_line[0], feature_max_vertical_line[1],  feature_max_vertical_line[2],  colors='black')
        ax[i, 0].set_xlabel('Wavelength ($\AA$)')
        ax[i, 0].set_ylabel(flux_units)
        #ax[i, 0].legend();
        
    for i in np.arange(num_rows, num_plots):
        #Plotting_data: 'cut_spectrum', 'norm_cont_line', 'left_knot_point', 'right_knot_point'
        cut_spectrum = plot_data[i]['cut_spectrum']   
        norm_cont_line = plot_data[i]['norm_cont_line'] 
        left_knot_point = plot_data[i]['left_knot_point']
        right_knot_point = plot_data[i]['right_knot_point']
        feature_min_vertical_line = [feature_min, cut_spectrum['y'].min(), cut_spectrum['y'].max()]
        feature_max_vertical_line = [feature_max, cut_spectrum['y'].min(), cut_spectrum['y'].max()]
        
        if model == True:
            ax[i-num_rows, 1].set_title(r'T = {}, EQW $\approx$ {}'.format(temp[i], np.round(eqw[i],0)))
        else:
            ax[i-num_rows, 1].set_title(r'Date = {}, EQW $\approx$ {}'.format(epochs[i].jd, np.round(eqw[i],0)))
        
        ax[i-num_rows, 1].plot(cut_spectrum['x'], cut_spectrum['y'], drawstyle='steps-mid', label='Initial Spectrum');  
        ax[i-num_rows, 1].plot(norm_cont_line['x'], norm_cont_line['y'], label = 'Continuum Fit')
        ax[i-num_rows, 1].scatter(left_knot_point['x'], left_knot_point['y'],  c = 'crimson', s = 100, label = 'Knot Points')
        ax[i-num_rows, 1].scatter(left_knot_point['x'], left_knot_point['y'], c = 'crimson', s = 200)
        ax[i-num_rows, 1].scatter(right_knot_point['x'], right_knot_point['y'], c = 'crimson', s = 200)
        ax[i-num_rows, 1].vlines(feature_min_vertical_line[0], feature_min_vertical_line[1],  feature_min_vertical_line[2], colors='black', label = 'Feature Endpoints')
        ax[i-num_rows, 1].vlines(feature_max_vertical_line[0], feature_max_vertical_line[1],  feature_max_vertical_line[2],  colors='black')
        ax[i-num_rows, 1].set_xlabel('Wavelength ($\AA$)')
        ax[i-num_rows, 1].set_ylabel(flux_units)
        #ax[i-num_rows, 1].legend();
        
    ax[np.arange(num_rows).max(), 0].set_xlabel('Wavelength ($\AA$)')
    ax[np.arange(num_rows).max(), 1].set_xlabel('Wavelength ($\AA$)')

    fig.tight_layout()
    
    #if model == True:
       # fig.suptitle(plot_data[0]['title'])
    
    
    if save_fig == True:
        plt.savefig(output_path + knot_plot_filename, bbox_inches='tight', aspect='auto')
        plt.close(fig)
        return knot_plot_filename
        
    else:
        fig.show() 


In [11]:
#Temperature Model Input: 
#                      Model Output data, OBS Output data, Input_params, EQW/Line_Ratio

#Temperature Model Output: 
#                      Epoch, Temp
def second(x, a, b, f):
    return (a * x) + (b * x**2) + f

def third(x, a, b, c, f):
    return (a * x) + (b * x**2) + (c * x**3) + f

def fourth(x, a, b, c, d, f):
    return (a * x) + (b * x**2) + (c * x**3) + (d * x**4) + f

def fifth(x, a, b, c, d, e, f):
    return (a * x) + (b * x**2) + (c * x**3) + (d * x**4) + (e*x**5) + f


def model_data_curve_fits_temp(model_output_data, obs_output_data,  line_selection, line_dict, output_path, save_fig = True):
    
    ###INPUT###
    
    #Input Params: absorption line, feature_min, feature_max, 
    #              left_cont_min, left_cont_max, 
    #              right_cont_min, right_cont_max
    
    absorption_line = line_selection
    selected_absorption_line = str(line_selection)
    
    input_params = line_dict[selected_absorption_line]
    
    feature_min = input_params['feature_min']
    feature_max =input_params['feature_max']
    
    left_cont_min, left_cont_max   = input_params['left_cont_min'],  input_params['left_cont_max']
    right_cont_min, right_cont_max = input_params['right_cont_min'], input_params['right_cont_max']  
        
    ####

    
    #output_data: 'EQW', 'plotting_data', 'epochs', ('temperatures')
    model_data = model_output_data['data']
    eqw_model = model_data['EQW']
    plot_data_model = model_output_data['plotting_data']
    temp = model_data['Teff']
    
    obs_data = obs_output_data['data']
    eqw_obs = obs_data['EQW']
    plot_data_obs = obs_output_data['plotting_data']
    
    


    xdata = eqw_model * u.AA/ u.AA
    ydata = temp * u.K/ u.K

    fit_xvals = np.linspace(min(xdata), max(xdata), 1000)

    
    
    obs_xdata = eqw_obs * u.AA/ u.AA
    
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(30, 20))
    
    popt, pcov = curve_fit(second, xdata, ydata)
    a, b, f = popt
    second_x = fit_xvals
    second_y = second(second_x, a, b, f)
    
    
    temps_second = np.zeros(len(obs_xdata))*u.K
    for i in np.arange(len(obs_xdata)):
        obs_ydata = second(obs_xdata[i], a, b, f)
        temps_second[i] = obs_ydata*u.K
        
    
    axs[0, 0].plot(second_x, second_y, 'r-',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))
    axs[0, 0].scatter(xdata, ydata, label='model data')
    axs[0, 0].scatter(obs_xdata, temps_second, label=r'T $\approx$ {}'.format(np.round(temps_second, 0)))
    axs[0, 0].set_title('Second Degree Polynomial')
    plt.grid()
    axs[0, 0].legend()
    
    popt, pcov = curve_fit(third, xdata, ydata)
    a, b, c, f = popt
    third_x = fit_xvals
    third_y = third(third_x, a, b, c, f)
    
        
    temps_third = np.zeros(len(obs_xdata))*u.K
    for i in np.arange(len(obs_xdata)):
        obs_ydata = third(obs_xdata[i], a, b, c, f)
        temps_third[i] = obs_ydata*u.K
    
    axs[0, 1].plot(third_x, third_y, 'b-',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f, d=%5.3f' % tuple(popt))
    axs[0, 1].scatter(xdata, ydata, label='model data')
    axs[0, 1].scatter(obs_xdata, temps_third, label=r'T $\approx$ {}'.format(np.round(temps_third, 0)))    
    axs[0, 1].set_title('Third Degree Polynomial')
    plt.grid()
    axs[0, 1].legend()
    
    if len(xdata) > 4:
        popt, pcov = curve_fit(fourth, xdata, ydata)
        a, b, c, d, f = popt
        fourth_x = fit_xvals
        fourth_y = fourth(fourth_x, a, b, c, d, f)

        temps_fourth = np.zeros(len(obs_xdata))*u.K
        for i in np.arange(len(obs_xdata)):
            obs_ydata = fourth(obs_xdata[i], a, b, c, d, f)
            temps_fourth[i] = obs_ydata*u.K

        axs[1, 0].plot(fourth_x, fourth_y, 'k-',
             label='fit: a=%5.3f, b=%5.3f, c=%5.3f, d=%5.3f, e=%5.3f' % tuple(popt))
        axs[1, 0].scatter(xdata, ydata, label='model data')
        axs[1, 0].scatter(obs_xdata, temps_fourth, label=r'T $\approx$ {}'.format(np.round(temps_fourth, 0)))
        axs[1, 0].set_title('Fourth Degree Polynomial')
        plt.grid()
        axs[1, 0].legend()

        popt, pcov = curve_fit(fifth, xdata, ydata)
        a, b, c, d, e, f = popt
        fifth_x = fit_xvals
        fifth_y = fifth(fifth_x, a, b, c, d, e, f)

        temps_fifth = np.zeros(len(obs_xdata))*u.K
        for i in np.arange(len(obs_xdata)):
            obs_ydata = fifth(obs_xdata[i], a, b, c, d, e, f)
            temps_fifth[i] = obs_ydata*u.K

        plt.plot(fifth_x, fifth_y, 'g-',
             label='fit: a=%5.3f, b=%5.3f, c=%5.3f, d=%5.3f, e=%5.3f, f=%5.3f' % tuple(popt))
        axs[1, 1].scatter(xdata, ydata, label='model data')
        axs[1, 1].scatter(obs_xdata, temps_fifth, label=r'T $\approx$ {}'.format(np.round(temps_fifth, 0)))
        axs[1, 1].set_title('Fifth Degree Polynomial')
        plt.grid()
        axs[1, 1].legend()
    else:
        temps_fourth = np.zeros(len(obs_xdata))*u.K
        temps_fifth = np.zeros(len(obs_xdata))*u.K
        
    
   
    for ax in axs.flat:
        ax.set(xlabel=r'EQW ($\AA$) ', ylabel=r'Temperature ($K$)')
    

# Hide x labels and tick labels for top plots and y ticks for right plots.
    for ax in axs.flat:
        ax.label_outer()
    filename = 'Model_Obs_Poly_fit_' + selected_absorption_line + ".pdf"

    if save_fig == True:
            plt.savefig(output_path + filename, bbox_inches='tight')
            plt.close()

    #plt.legend()
    plt.show()
    return temps_second, temps_third, temps_fourth, temps_fifth, filename

In [12]:
def EQW_plot(output_data, line_selection, line_dict, output_path, nc = 2, model = False, save_fig = False):
    flux_units = (u.erg)*(u.cm**(-2))*(u.s**(-1))*(u.AA**(-1))
    

    
    ###INPUT###
    
    #Input Params: absorption line, feature_min, feature_max, 
    #              left_cont_min, left_cont_max, 
    #              right_cont_min, right_cont_max
    
    absorption_line = line_selection
    selected_absorption_line = str(line_selection)
    
    input_params = line_dict[selected_absorption_line]
    
    feature_min = input_params['feature_min']
    feature_max =input_params['feature_max']
    feature_midpoint = (feature_min + feature_max)/2
    
    left_cont_min, left_cont_max   = input_params['left_cont_min'],  input_params['left_cont_max']
    right_cont_min, right_cont_max = input_params['right_cont_min'], input_params['right_cont_max']  

    #output_data: 'EQW', 'plotting_data', 'epochs', ('temperatures')
    data = output_data['data']
    eqw = data['EQW']
    plot_data = output_data['plotting_data']
    
    
    if model == False:
        epochs = output_data['epoch']
        eqw_plot_filename = 'Obs_eqw_plot_' + selected_absorption_line + ".pdf"
        selected_color = 'maroon'
    elif model == True:
        eqw_plot_filename = 'Model_eqw_plot_' + selected_absorption_line + ".pdf"
        temp = data['Teff']
        selected_color = 'navy'
    
    num_plots = len(output_data['plotting_data'])
    
    if (num_plots % 2) == 0:
        num_rows = math.floor(num_plots/nc)
    else:
        num_rows = math.floor(num_plots/nc + (num_plots)%nc)
    
    num_cols = nc
    
    fig, ax = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(30, 7*num_rows))
    #trans = mtransforms.blended_transform_factory(ax.transData, ax.transAxes)
    if model == False:
        fig.suptitle('EQW Observed Data')
    elif model == True:
        fig.suptitle('EQW Model Data')
    for i in np.arange(num_rows):
        #Plotting_data: 'cut_spectrum', 'norm_cont_line', 'left_knot_point', 'right_knot_point'
        cut_spectrum = plot_data[i]['cut_spectrum']   
        norm_cont_line = plot_data[i]['norm_cont_line'] 
        feature_cut_spectrum = plot_data[i]['feature_cut_spectrum']   
        feature_norm_cont_line = plot_data[i]['feature_norm_cont_line'] 
        left_knot_point = plot_data[i]['left_knot_point']
        right_knot_point = plot_data[i]['right_knot_point']
        feature_min_vertical_line = [feature_min, cut_spectrum['y'].min(), cut_spectrum['y'].max()]
        feature_max_vertical_line = [feature_max, cut_spectrum['y'].min(), cut_spectrum['y'].max()]
        feature_cut = [(cut_spectrum['x'] >= feature_min) & (cut_spectrum['x'] <= feature_max)]
        norm_cut_spec_y = cut_spectrum['y']/norm_cont_line['y']
        norm_cut_spec_y_feat = cut_spectrum['y'][feature_cut]/norm_cont_line['y'][feature_cut]
        
        if model == True:
            ax[i, 0].set_title(r'T = {}, EQW $\approx$ {}'.format(temp[i], np.round(eqw[i],0)))
        else:
            ax[i, 0].set_title(r'Date = {}, EQW $\approx$ {}'.format(epochs[i].jd, np.round(eqw[i],0)))
            

        ax[i, 0].plot(cut_spectrum['x'], cut_spectrum['y']/norm_cont_line['y'], drawstyle='steps-mid', label='Normalized Spectrum');  
        ax[i, 0].plot(norm_cont_line['x'], norm_cont_line['y']/norm_cont_line['y'],'--r', label = 'Continuum Fit')
        ax[i, 0].vlines(feature_cut_spectrum['x'].min(),  0,  1.0, colors='black', label = 'Feature Endpoints')
        ax[i, 0].vlines(feature_cut_spectrum['x'].max(), 0,  1.0,  colors='black')
        

        ax[i, 0].fill_between(feature_cut_spectrum['x'], 1, norm_cut_spec_y_feat, 
                               where=norm_cut_spec_y_feat < 1, facecolor='green', interpolate=True, alpha=0.3)
        ax[i, 0].add_patch(plt.Rectangle(((feature_midpoint)-(eqw[i]/2), 0.0), eqw[i],1.0, ec='k', fc="k",
                            alpha=0.3))
        ax[i, 0].set_xlabel('Wavelength ($\AA$)')
        ax[i, 0].set_ylabel('Normalized Flux')
        #ax[i, 0].legend();
        
    for i in np.arange(num_rows, num_plots):
        #Plotting_data: 'cut_spectrum', 'norm_cont_line', 'left_knot_point', 'right_knot_point'
        cut_spectrum = plot_data[i]['cut_spectrum']   
        norm_cont_line = plot_data[i]['norm_cont_line'] 
        feature_cut_spectrum = plot_data[i]['feature_cut_spectrum']   
        feature_norm_cont_line = plot_data[i]['feature_norm_cont_line'] 
        left_knot_point = plot_data[i]['left_knot_point']
        right_knot_point = plot_data[i]['right_knot_point']
        feature_min_vertical_line = [feature_min, cut_spectrum['y'].min(), cut_spectrum['y'].max()]
        feature_max_vertical_line = [feature_max, cut_spectrum['y'].min(), cut_spectrum['y'].max()]
        feature_cut = [(cut_spectrum['x'] >= feature_min) & (cut_spectrum['x'] <= feature_max)]
        norm_cut_spec_y = cut_spectrum['y']/norm_cont_line['y']
        norm_cut_spec_y_feat = cut_spectrum['y'][feature_cut]/norm_cont_line['y'][feature_cut]

        if model == True:
            ax[i-num_rows, 1].set_title(r'T = {}, EQW $\approx$ {}'.format(temp[i], np.round(eqw[i],0)))
        else:
            ax[i-num_rows, 1].set_title(r'Date = {}, EQW $\approx$ {}'.format(epochs[i].jd, np.round(eqw[i],0)))
        
        ax[i-num_rows, 1].plot(cut_spectrum['x'], norm_cut_spec_y, drawstyle='steps-mid', label='Normalized Spectrum');  
        ax[i-num_rows, 1].plot(norm_cont_line['x'], norm_cont_line['y']/norm_cont_line['y'], '--r', label = 'Continuum Fit')
        ax[i-num_rows, 1].vlines(feature_cut_spectrum['x'].min(),  0,  1.0, colors='black', label = 'Feature Endpoints')
        ax[i-num_rows, 1].vlines(feature_cut_spectrum['x'].max(), 0,  1.0,  colors='black')
        

        ax[i-num_rows, 1].fill_between(feature_cut_spectrum['x'], 1, norm_cut_spec_y_feat, 
                               where=norm_cut_spec_y_feat < 1, facecolor='green', interpolate=True, alpha=0.3)
        ax[i-num_rows, 1].add_patch(plt.Rectangle(((feature_midpoint)-(eqw[i]/2), 0.0), eqw[i],1.0, ec='k', fc="k",
                            alpha=0.3))
        ax[i-num_rows, 1].set_xlabel('Wavelength ($\AA$)')
        ax[i-num_rows, 1].set_ylabel('Normalized Flux')
        #ax[i-num_rows, 1].legend();
        
    ax[np.arange(num_rows).max(), 0].set_xlabel('Wavelength ($\AA$)')
    ax[np.arange(num_rows).max(), 1].set_xlabel('Wavelength ($\AA$)')

    fig.tight_layout()
    
    #if model == True:
       # fig.suptitle(plot_data[0]['title'])
    
    
    if save_fig == True:
        plt.savefig(output_path + eqw_plot_filename, bbox_inches='tight', aspect='auto')
        plt.close(fig)
        return eqw_plot_filename
        
    else:
        fig.show() 


def EQW_plot_feat_only(output_data, line_selection, line_dict, output_path, nc = 2, model = False, save_fig = False):
    flux_units = (u.erg)*(u.cm**(-2))*(u.s**(-1))*(u.AA**(-1))
    

    
    ###INPUT###
    
    #Input Params: absorption line, feature_min, feature_max, 
    #              left_cont_min, left_cont_max, 
    #              right_cont_min, right_cont_max
    
    absorption_line = line_selection
    selected_absorption_line = str(line_selection)
    
    input_params = line_dict[selected_absorption_line]
    
    feature_min = input_params['feature_min']
    feature_max =input_params['feature_max']
    feature_midpoint = (feature_min + feature_max)/2
    
    left_cont_min, left_cont_max   = input_params['left_cont_min'],  input_params['left_cont_max']
    right_cont_min, right_cont_max = input_params['right_cont_min'], input_params['right_cont_max']  

    #output_data: 'EQW', 'plotting_data', 'epochs', ('temperatures')
    data = output_data['data']
    eqw = data['EQW']
    plot_data = output_data['plotting_data']
    
    
    if model == False:
        epochs = output_data['epoch']
        eqw_plot_filename = 'Obs_eqw_plot_' + selected_absorption_line + ".pdf"
        selected_color = 'maroon'
    elif model == True:
        eqw_plot_filename = 'Model_eqw_plot_' + selected_absorption_line + ".pdf"
        temp = data['Teff']
        selected_color = 'navy'
    
    num_plots = len(output_data['plotting_data'])
    
    if (num_plots % 2) == 0:
        num_rows = math.floor(num_plots/nc)
    else:
        num_rows = math.floor(num_plots/nc + (num_plots)%nc)
    
    num_cols = nc
    
    fig, ax = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(30, 7*num_rows))
    #trans = mtransforms.blended_transform_factory(ax.transData, ax.transAxes)
    if model == False:
        fig.suptitle('EQW Observed Data')
    elif model == True:
        fig.suptitle('EQW Model Data')
    
    
    for i in np.arange(num_rows):
        #Plotting_data: 'cut_spectrum', 'norm_cont_line', 'left_knot_point', 'right_knot_point'
        cut_spectrum = plot_data[i]['cut_spectrum']   
        norm_cont_line = plot_data[i]['norm_cont_line'] 
        feature_cut_spectrum = plot_data[i]['feature_cut_spectrum']   
        feature_norm_cont_line = plot_data[i]['feature_norm_cont_line'] 
        left_knot_point = plot_data[i]['left_knot_point']
        right_knot_point = plot_data[i]['right_knot_point']
        feature_min_vertical_line = [feature_min, cut_spectrum['y'].min(), cut_spectrum['y'].max()]
        feature_max_vertical_line = [feature_max, cut_spectrum['y'].min(), cut_spectrum['y'].max()]
        
        if model == True:
            ax[i, 0].set_title(r'T = {}, EQW $\approx$ {}'.format(temp[i], np.round(eqw[i],0)))
        else:
            ax[i, 0].set_title(r'Date = {}, EQW $\approx$ {}'.format(epochs[i].jd, np.round(eqw[i],0)))
            

        ax[i, 0].plot(feature_cut_spectrum['x'], feature_cut_spectrum['y'], drawstyle='steps-mid', label='Initial Spectrum');  
        ax[i, 0].plot(feature_norm_cont_line['x'], feature_norm_cont_line['y'], label = 'Continuum Fit')
        ax[i, 0].vlines(feature_cut_spectrum['x'].min(),  0,  1.0, colors='black', label = 'Feature Endpoints')
        ax[i, 0].vlines(feature_cut_spectrum['x'].max(), 0,  1.0,  colors='black')
        

        ax[i, 0].fill_between(feature_cut_spectrum['x'], 1, feature_cut_spectrum['y'], 
                               where=feature_cut_spectrum['y'] < 1, facecolor='green', interpolate=True, alpha=0.3)
        ax[i, 0].add_patch(plt.Rectangle(((feature_midpoint)-(eqw[i]/2), 0.0), eqw[i],1.0, ec='k', fc="k",
                            alpha=0.3))
        ax[i, 0].set_xlabel('Wavelength ($\AA$)')
        ax[i, 0].set_ylabel('Normalized Flux')
        #ax[i, 0].legend();
        
    for i in np.arange(num_rows, num_plots):
        #Plotting_data: 'cut_spectrum', 'norm_cont_line', 'left_knot_point', 'right_knot_point'
        cut_spectrum = plot_data[i]['cut_spectrum']   
        norm_cont_line = plot_data[i]['norm_cont_line'] 
        left_knot_point = plot_data[i]['left_knot_point']
        right_knot_point = plot_data[i]['right_knot_point']
        feature_min_vertical_line = [feature_min, cut_spectrum['y'].min(), cut_spectrum['y'].max()]
        feature_max_vertical_line = [feature_max, cut_spectrum['y'].min(), cut_spectrum['y'].max()]
        
        if model == True:
            ax[i-num_rows, 1].set_title(r'T = {}, EQW $\approx$ {}'.format(temp[i], np.round(eqw[i],0)))
        else:
            ax[i-num_rows, 1].set_title(r'Date = {}, EQW $\approx$ {}'.format(epochs[i].jd, np.round(eqw[i],0)))
        
        ax[i-num_rows, 1].plot(feature_cut_spectrum['x'], feature_cut_spectrum['y'], drawstyle='steps-mid', label='Initial Spectrum');  
        ax[i-num_rows, 1].plot(feature_norm_cont_line['x'], feature_norm_cont_line['y'], label = 'Continuum Fit')
        ax[i-num_rows, 1].vlines(feature_cut_spectrum['x'].min(),  0,  1.0, colors='black', label = 'Feature Endpoints')
        ax[i-num_rows, 1].vlines(feature_cut_spectrum['x'].max(), 0,  1.0,  colors='black')
        ax[i-num_rows, 1].fill_between(feature_cut_spectrum['x'], 1, feature_cut_spectrum['y'], 
                               where=feature_cut_spectrum['y'] < 1, facecolor='green', interpolate=True, alpha=0.3)
        ax[i-num_rows, 1].add_patch(plt.Rectangle(((feature_midpoint)-(eqw[i]/2), 0.0), eqw[i],1.0, ec='k', fc="k",
                            alpha=0.3))
        ax[i-num_rows, 1].set_xlabel('Wavelength ($\AA$)')
        ax[i-num_rows, 1].set_ylabel('Normalized Flux')
        #ax[i-num_rows, 1].legend();
        
    ax[np.arange(num_rows).max(), 0].set_xlabel('Wavelength ($\AA$)')
    ax[np.arange(num_rows).max(), 1].set_xlabel('Wavelength ($\AA$)')

    fig.tight_layout()
    
    #if model == True:
       # fig.suptitle(plot_data[0]['title'])
    
    
    if save_fig == True:
        plt.savefig(output_path + eqw_plot_filename, bbox_inches='tight', aspect='auto')
        plt.close(fig)
        return eqw_plot_filename
        
    else:
        fig.show() 


In [13]:
def EQW_Vs_Temp_Scatter(Model_output_data, Obs_output_data, line_selection, line_dict, output_path, save_fig = False, plot_axes = 'TEMP_EQW'):
    flux_units = (u.erg)*(u.cm**(-2))*(u.s**(-1))*(u.AA**(-1))
    

    
    ###INPUT###
    
    #Input Params: absorption line, feature_min, feature_max, 
    #              left_cont_min, left_cont_max, 
    #              right_cont_min, right_cont_max
    
    absorption_line = line_selection
    selected_absorption_line = str(line_selection)
    
    input_params = line_dict[selected_absorption_line]
    
    feature_min = input_params['feature_min']
    feature_max =input_params['feature_max']
    feature_midpoint = (feature_min + feature_max)/2
    
    left_cont_min, left_cont_max   = input_params['left_cont_min'],  input_params['left_cont_max']
    right_cont_min, right_cont_max = input_params['right_cont_min'], input_params['right_cont_max']  
    
    model_data = Model_output_data['data']
    eqw_model = model_data['EQW']
    temp = model_data['Teff']
    
    obs_data = Obs_output_data['data']
    eqw_obs = obs_data['EQW']
    plot_data_obs = Obs_output_data['plotting_data']
    
    Epochs = Obs_output_data['epoch']
    
    EQW_v_Temp = 'EQW_v_Temp_' + selected_absorption_line + ".pdf"
    colors = ['b', 'g', 'r', 'k', 'c', 'm', 'y']
    linestyles = ['-', '--', '-.', ':']
    
    
    if plot_axes == 'TEMP_EQW':
        fig, (ax1, ax2) = plt.subplots(nrows= 1, ncols= 2)
        fig.set_size_inches(20, 10)
        fig.suptitle(r'EQW: {} ($\AA$) vs. Temperature'.format(absorption_line))
        ax1.scatter(temp, eqw_model)

        for i in range(len(Epochs)):
            ax1.axhline(y=eqw_obs[i], color = '{}'.format(colors[i]), lw=2, ls = linestyles[i], label = r'EQW: {} $\AA$, {}'.format(absorption_line, Epochs[i]))
        ax1.set_ylabel(r'$EQW \ (\AA)$')
        ax1.set_xlabel(r'$T_{eff} \ (K)$')
        ax1.legend()

        ax2.scatter(temp, np.log(eqw_model))
        for i in range(len(Epochs)):
            ax2.axhline(y=np.log(eqw_obs[i]), color = '{}'.format(colors[i]), lw=2, ls = linestyles[i], label = r'EQW: {} $\AA$, {}'.format(absorption_line, Epochs[i]))
        ax2.set_ylabel(r'$log(EQW \ (\AA))$')
        ax2.set_xlabel(r'$T_{eff} \ (K)$')
        ax2.legend()    

        if save_fig == True:
            plt.savefig(output_path + EQW_v_Temp, bbox_inches='tight', aspect='auto')
            plt.close(fig)
            return EQW_v_Temp

        else:
            fig.show()
    
    elif plot_axes == 'EQW_TEMP':
        fig, (ax1, ax2) = plt.subplots(nrows= 1, ncols= 2)
        fig.set_size_inches(20, 10)
        fig.suptitle(r'EQW: {} ($\AA$) vs. Temperature'.format(absorption_line))
        ax1.scatter(eqw_model, temp)

        for i in range(len(Epochs)):
            ax1.axvline(x=eqw_obs[i], color = '{}'.format(colors[i]), lw=2, ls = linestyles[i], label = r'EQW: {} $\AA$, {}'.format(absorption_line, Epochs[i]))
        ax1.set_xlabel(r'$EQW \ (\AA)$')
        ax1.set_ylabel(r'$T_{eff} \ (K)$')
        ax1.legend()

        ax2.scatter(np.log(eqw_model), temp)
        for i in range(len(Epochs)):
            ax2.axvline(x=np.log(eqw_obs[i]), color = '{}'.format(colors[i]), lw=2, ls = linestyles[i], label = r'EQW: {} $\AA$, {}'.format(absorption_line, Epochs[i]))
        ax2.set_xlabel(r'$log(EQW \ (\AA))$')
        ax2.set_ylabel(r'$T_{eff} \ (K)$')
        ax2.legend()    

        if save_fig == True:
            plt.savefig(output_path + EQW_v_Temp, bbox_inches='tight', aspect='auto')
            plt.close(fig)
            return EQW_v_Temp

        else:
            fig.show()


In [14]:
def Export_EQW_plots(Model_output_data, Obs_output_data, line_selection, line_dict, output_path):
    
    flux_units = (u.erg)*(u.cm**(-2))*(u.s**(-1))*(u.AA**(-1))
    

    
    ###INPUT###
    
    #Input Params: absorption line, feature_min, feature_max, 
    #              left_cont_min, left_cont_max, 
    #              right_cont_min, right_cont_max
    
    absorption_line = line_selection
    selected_absorption_line = str(line_selection)
    
    input_params = line_dict[selected_absorption_line]
    
    feature_min = input_params['feature_min']
    feature_max =input_params['feature_max']
    feature_midpoint = (feature_min + feature_max)/2
    
    left_cont_min, left_cont_max   = input_params['left_cont_min'],  input_params['left_cont_max']
    right_cont_min, right_cont_max = input_params['right_cont_min'], input_params['right_cont_max']  
        
    ####
    Input_Param_Table = input_param_table(line_dict, absorption_line, output_path, save_fig = True)
    Model_knot_plot = knot_plot(Model_output_data, absorption_line, line_dict, output_path, nc = 2, model = True, save_fig = True)
    Model_EQW_plot = EQW_plot(Model_output_data, absorption_line, line_dict, output_path, nc = 2, model = True, save_fig = True)

    Obs_knot_plot = knot_plot(Obs_output_data, absorption_line, line_dict, output_path, nc = 2, model = False, save_fig = True)
    Obs_EQW_plot = EQW_plot(Obs_output_data, absorption_line, line_dict, output_path, nc = 2, model = False, save_fig = True)

    EQW_v_Temp = EQW_Vs_Temp_Scatter(Model_output_data, Obs_output_data, absorption_line, line_dict, output_path, save_fig = True)
    
    pdfs = [output_path + Input_Param_Table,
            output_path + Model_knot_plot,
            output_path + Model_EQW_plot,
            output_path + Obs_knot_plot,
            output_path + Obs_EQW_plot,
            output_path + EQW_v_Temp]
    
    merger = PdfFileMerger()

    for pdf in pdfs:
        merger.append(pdf)

    merger.write(output_path + str(absorption_line)+ "_All_Plots.pdf")
    merger.close()
    
    print('{} complete'.format(selected_absorption_line))


In [15]:
def Export_EQW_and_temp_plots(Model_output_data, Obs_output_data, line_selection, line_dict, output_path):
    
    flux_units = (u.erg)*(u.cm**(-2))*(u.s**(-1))*(u.AA**(-1))
    

    
    ###INPUT###
    
    #Input Params: absorption line, feature_min, feature_max, 
    #              left_cont_min, left_cont_max, 
    #              right_cont_min, right_cont_max
    
    absorption_line = line_selection
    selected_absorption_line = str(line_selection)
    
    input_params = line_dict[selected_absorption_line]
    
    feature_min = input_params['feature_min']
    feature_max =input_params['feature_max']
    
    left_cont_min, left_cont_max   = input_params['left_cont_min'],  input_params['left_cont_max']
    right_cont_min, right_cont_max = input_params['right_cont_min'], input_params['right_cont_max']  
        
    ####
    Input_Param_Table = input_param_table(line_dict, line_selection, output_path, save_fig = True)
    Model_knot_plot = knot_plot(Model_output_data, absorption_line, line_dict, output_path, nc = 2, model = True, save_fig = True)
    Model_EQW_plot = EQW_plot(Model_output_data, absorption_line, line_dict, output_path, nc = 2, model = True, save_fig = True)

    Obs_knot_plot = knot_plot(Obs_output_data, absorption_line, line_dict, output_path, nc = 2, model = False, save_fig = True)
    Obs_EQW_plot = EQW_plot(Obs_output_data, absorption_line, line_dict, output_path, nc = 2, model = False, save_fig = True)

    
    temps_second, temps_third, temps_fourth, temps_fifth, Poly_plot = model_data_curve_fits_temp(Model_output_data, Obs_output_data, absorption_line, line_dict, output_path, save_fig = True)
    Columns = ['Epoch', r'$T_{eff}$, deg: 2', r'$T_{eff}$, deg: 3', r'$T_{eff}$, deg: 4', r'$T_{eff}$, deg: 5']
    Epochs = Obs_output_data['epoch']
    df = pd.DataFrame(list(zip(Epochs, 
                               np.round(temps_second, 2), 
                               np.round(temps_third, 2), 
                               np.round(temps_fourth, 2),
                               np.round(temps_fifth, 2))),
               columns = Columns)
    df.round(1)
    
    fig, ax = plt.subplots()
    fig.patch.set_visible(False)

    ax.axis('off')
    ax.axis('tight')

    Temp_table = 'Temp_Fits' + selected_absorption_line + ".pdf"

    #create table
    table = ax.table(cellText=df.values, colLabels=Columns, loc='center', label = r'{} $\AA$'.format(absorption_line))

    #modify table
    table.set_fontsize(14)
    table.scale(1,4)
    #ax.set_title(r'{} $\AA$'.format(absorption_line), y=0.75, pad=-14)
    #ax.set_xlabel(r'{} $\AA$'.format(absorption_line))
    #ax.xaxis.set_label_position('top')
    #ax.legend()
    
    
    plt.savefig(output_path + Temp_table, bbox_inches='tight', aspect='auto')
    plt.close(fig)
    
    
    pdfs = [output_path + Input_Param_Table,
            output_path + Model_knot_plot,
            output_path + Model_EQW_plot,
            output_path + Obs_knot_plot,
            output_path + Obs_EQW_plot,
            output_path + Poly_plot,
            output_path + Temp_table]
    
    merger = PdfFileMerger()

    for pdf in pdfs:
        merger.append(pdf)

    merger.write(output_path + str(absorption_line)+ "_All_Plots.pdf")
    merger.close()
    
    
    

In [16]:
def EQW_df(model_output_dict):
    df = pd.DataFrame.from_dict(model_output_dict)
    return df


In [17]:
def input_param_table(line_dict, line_selection, output_path, save_fig = False):
    absorption_line = line_selection
    selected_absorption_line = str(line_selection)
    
    input_params = line_dict[selected_absorption_line]
    
    columns = ['feature min', 'feature max', 
           'left cont min',' left cont max',
           'right cont min',' right cont max']

    fig, ax = plt.subplots()
    fig.patch.set_visible(False)

    ax.axis('off')
    ax.axis('tight')
    df = pd.DataFrame.from_dict([input_params])

    table_filename = 'Input_param' + selected_absorption_line + ".pdf"

    #create table
    table = ax.table(cellText=df.values, colLabels=columns, loc='center', label = r'{} $\AA$'.format(absorption_line))

    #modify table
    table.set_fontsize(14)
    table.scale(1,4)
    ax.set_title(r'{} $\AA$'.format(absorption_line), y=0.75, pad=-14)
    #ax.set_xlabel(r'{} $\AA$'.format(absorption_line))
    #ax.xaxis.set_label_position('top')
    #ax.legend()
    
    if save_fig == True:
        plt.savefig(output_path + table_filename, bbox_inches='tight', aspect='auto')
        plt.close(fig)
        return table_filename

    else:
        plt.show()

In [18]:
def return_aavso_data(aavso_data_path):
    df = pd.read_csv(aavso_data_path, header=None)
    df = df[[0,1,2,4]]
    df.columns = ['time', 'mag', 'error', 'band']
    return df